<a href="https://colab.research.google.com/github/Nayan-Bebale/object-detection-models-hub/blob/main/SSD_Mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SSD-Mobilenet Model

## Step 1
Install tensorflow version 2 or higher

In [ ]:
!pip install -U --pre tensorflow=="2.*"

## Step 2
make sure to install pycocotools for coco detection API.

In [ ]:
!pip install pycocotools

## Step 3
get tensorflow/models by cloning the repository.

In [ ]:
import os
import pathlib

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

move (cd) to research directory of the repo

In [ ]:
cd models/research

## Step 4
compile protobufs

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

install object_detection python package

In [ ]:
!pip install object_detection

## Step 5
 import required libraries

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

install tf_slim python package:

In [ ]:
!pip install tf_slim

import object detection modules:

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

## step 6
function to load your model

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model

In [ ]:
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

## step 7
load your object detection SSD mobilenet v1 model for object detection

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

In [ ]:
print(detection_model.inputs)
detection_model.output_dtypes

In [ ]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis, ...]

    output_dict = model(input_tensor)

    num_detections = int(output_dict.pop('num_detections'))

    output_dict = {
        key: value[0, :num_detections].numpy()
        for key, value in output_dict.items()
    }

    output_dict['num_detections'] = num_detections
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict['detection_masks'],
            output_dict['detection_boxes'],
            image.shape[0],
            image.shape[1]
        )

        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict

## step 8

In [ ]:
def show_inference(model, image_path, class_id):
    image_np = np.array(Image.open(image_path))
    output_dict = run_inference_for_single_image(model, image_np)

    boxes = []
    classes = []
    scores = []

    for i, x in enumerate(output_dict['detection_classes']):
        if x == class_id and output_dict['detection_scores'][i] > 0.5:
            classes.append(x)
            boxes.append(output_dict['detection_boxes'][i])
            scores.append(output_dict['detection_scores'][i])

    boxes = np.array(boxes)
    classes = np.array(classes)
    scores = np.array(scores)

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        boxes,
        classes,
        scores,
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=2
    )

    display(Image.fromarray(image_np))

## step 9

the **class_id** in your code should represent a specific class of object (such as a person, car, etc.) from the COCO dataset, which contains 80 classes of common objects. You need to define class_id before using it in the show_inference function.

In [ ]:
class_id = 1  # For example, 1 represents 'person' in the COCO dataset
for image_path in TEST_IMAGE_PATHS:
    show_inference(detection_model, image_path, class_id)

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path, 18)

## Step 10

If you want to detect all objects in the image, you don't need to filter by class_id. You can modify the show_inference function to avoid filtering by any specific class. This way, it will display all detected objects with a confidence score above a certain threshold (e.g., 0.5).

In [ ]:
def show_inference_all(model, image_path):
    image_np = np.array(Image.open(image_path))
    output_dict = run_inference_for_single_image(model, image_np)

    boxes = []
    classes = []
    scores = []

    for i, score in enumerate(output_dict['detection_scores']):
        if score > 0.5:  # Only show objects with detection scores above the threshold
            classes.append(output_dict['detection_classes'][i])
            boxes.append(output_dict['detection_boxes'][i])
            scores.append(score)

    boxes = np.array(boxes)
    classes = np.array(classes)
    scores = np.array(scores)

    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        boxes,
        classes,
        scores,
        category_index,
        instance_masks=output_dict.get('detection_masks_reframed', None),
        use_normalized_coordinates=True,
        line_thickness=2
    )

    display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    show_inference_all(detection_model, image_path)  # No class_id needed